In [0]:
import pandas as pd
import pyproj
import geopandas as gpd
import matplotlib

import geopandas as gpd
from shapely import wkt


from shapely.geometry import Point
from shapely.wkt import loads

import unicodedata
import requests
import json
import os
import re
import numpy as np

from fuzzywuzzy import fuzz

import jenkspy

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, cast, when, udf, struct, from_json, count, countDistinct, lit
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType
from pyspark import StorageLevel

from sedona.spark import *
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.register.geo_registrator import SedonaRegistrator
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.utils.adapter import Adapter
from sedona.core.enums import IndexType
from sedona.core.enums import GridType
from sedona.core.spatialOperator import JoinQuery

from sedona.core.SpatialRDD import PointRDD



In [0]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
num_partitions = 4
url = "https://catalogopmb.catastrobogota.gov.co/PMBWeb/web/api"
startpath = '/dbfs/FileStore/tables/Catastro/'

In [0]:
usos_homologados={
    'Asociar al uso principal':['AREA DE MEZANINE EN PH','DEPOSITO (LOCKERS) PH','DEPOSITO ALMACENAMIENTO PH','PARQUEO CUBIERTO NPH','PARQUEO CUBIERTO PH','PARQUEO LIBRE PH','PISCINAS EN NPH','PISCINAS EN PH','SECADEROS'],
    'Comercio y servicios':['BODEGA COMERCIAL NPH','BODEGA COMERCIAL PH','BODEGA ECONOMICA','BODEGA ECONOMICA(SERVITECA,ESTA.SERVIC.)','BODEGAS DE ALMACENAMIENTO NPH','BODEGAS DE ALMACENAMIENTO PH','CENTRO COMERCIAL GRANDE NPH','CENTRO COMERCIAL GRANDE PH','CENTRO COMERCIAL MEDIANO NPH','CENTRO COMERCIAL MEDIANO PH','CENTRO COMERCIAL PEQUENO NPH','CENTRO COMERCIAL PEQUENO PH','CLUBES PEQUENOS','COMERCIO PUNTUAL NPH O HASTA 3 UNID PH','COMERCIO PUNTUAL PH','CORREDOR COMERCIAL NPH O HASTA 3 UNID PH','CORREDOR COMERCIAL PH','DEPOSITOS DE ALMACENAMIENTO NPH','EDIFICIOS DE PARQUEO NPH','EDIFICIOS DE PARQUEO PH','HOTELES NPH','HOTELES PH','MOTELES, AMOBLADOS, RESIDENCIAS NPH','MOTELES, AMOBLADOS, RESIDENCIAS PH','OFICINA BODEGA Y/O INDUSTRIA PH','OFICINAS EN BODEGAS Y/O INDUSTRIAS','OFICINAS OPERATIVAS','OFICINAS OPERATIVAS(ESTACIONES SERVICIO)','OFICINAS Y CONSULTORIOS NPH','OFICINAS Y CONSULTORIOS PH','PARQUES DE DIVERSION','RESTAURANTES NPH','RESTAURANTES PH','TEATROS Y CINEMAS NPH','TEATROS Y CINEMAS PH', 'PARQUES DE DIVERSION EN P.H.'],
    'Dotacional':['AULAS DE CLASE','CEMENTERIOS','CENTROS MEDICOS EN PH','CLINICAS, HOSPITALES, CENTROS MEDICOS','CLUBES MAYOR EXTENSION','COLEGIOS EN PH','COLEGIOS Y UNIVERSIDADES 1 A 3 PISOS','COLEGIOS Y UNIVERSIDADES 4 O MAS PISOS','COLISEOS','CULTO RELIGIOS EN NPH','CULTO RELIGIOSO EN PH','IGLESIA PH','IGLESIAS','INSTALACIONES MILITARES','INSTITUCIONAL PH','INSTITUCIONAL PUNTUAL','MUSEOS','OFICINAS Y CONSULTORIOS (OFICIAL) NPH','OFICINAS Y CONSULTORIOS (OFICIAL) PH','PLAZAS DE MERCADO'],
    'Industrial':['INDUSTRIA ARTESANAL','INDUSTRIA GRANDE','INDUSTRIA GRANDE PH','INDUSTRIA MEDIANA','INDUSTRIA MEDIANA PH'],
    'Otro':['COCHERAS, MARRANERAS, PORQUERIZAS','ENRAMADAS, COBERTIZOS, CANEYES','ESTABLOS, PESEBRERAS','GALPONES, GALLINEROS','KIOSKOS','LOTE EN PROPIEDAD HORIZONTAL','PISTA AEROPUERTO','SILOS'],
    'Residencial':['HABITACIONAL EN PROPIEDAD HORIZONTAL','HABITACIONAL MAYOR O IGUAL A 4 PISOS NPH O 3 PISOS PH','HABITACIONAL MENOR O IGUAL A 3 PISOS NPH','HABITACIONAL MENOR O IGUAL A 3 PISOS PH','MAYOR O IGUAL A 4 PISOS NPH O 3 PISOS PH']
 }

In [0]:
def remove_words_from_string(input_str, words_to_remove):
    for word in words_to_remove:
        input_str = input_str.replace(word, "")
    return input_str

In [0]:
def clean_ph_address(s):
    s = re.sub(r'\b(AP|GJ|IN|LT) \d+\b', '', s).strip()
    if 'SUR' in s:
        s = re.sub(r'\bSUR\b', '', s).strip() + ' S'
    return s

In [0]:
create_point_udf = udf(lambda x,y: Point(x,y).wkt, returnType=StringType())

In [0]:
def homologar_uso(uso_catastro:str,usos_agregados=usos_homologados):
    '''
    Toma el uso de la base de predios de catastro y lo agrega a 
    categorias mas sencillas de entender para el análisis de datos del OOVS

    Args:
        input_str (str): Cadena de texto que trae el uso para gregarlo.

    Returns:
        str: retorna el uso agragado y homologado por el equipo del observatorio
    '''
    for k,v in usos_agregados.items():
        if uso_catastro in v:
            return k
        elif uso_catastro not in v:
            pass
homoUso_udf = udf(homologar_uso, StringType())

In [0]:
def transform_coordinates(x, y):
    if x is not None and y is not None and x != 'NULL' and y!='NULL' and x != 0.0 and y!=0.0 and isinstance(x, (float)) and isinstance(y, (float)):
        source_crs = pyproj.CRS("ESRI:102233")
        target_crs = pyproj.CRS("EPSG:4326")
        transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
        new_x, new_y = transformer.transform(x, y)
        return new_x, new_y
    else:
        return None, None

coordinate_udf = udf(transform_coordinates, StructType([StructField("wgs84_lng", DoubleType(), False),
                                                        StructField("wgs84_lat", DoubleType(), False)]))

In [0]:
def get_year_from_path(path, paths_dict):
    # Iterate over the dictionary to find the year for the given path
    for year, paths in paths_dict.items():
        if path in paths:
            return year
    return None

In [0]:
def transform_record(record):
    localidad_parts = record['LOCALIDAD'].split()
    codigo_localidad = localidad_parts[0]
    nombre_localidad = ' '.join(localidad_parts[1:])
    
    tipo_propiedad_parts = str(record['TIPO_PROPIEDAD']).split()
    tipo_propiedad = tipo_propiedad_parts[0]
    descripcion_tipo_propiedad = ' '.join(tipo_propiedad_parts[1:])
    
    transformed = {
        '_c0':0,
        'CODIGO_LOCALIDAD': codigo_localidad,
        'NOMBRE_LOCALIDAD': nombre_localidad,
        'CODIGO_BARRIO': record['CODIGO_BARRIO'],
        'CODIGO_MANZANA': record['CODIGO_MANZANA'],
        'CODIGO_PREDIO': record['CODIGO_PREDIO'],
        'CODIGO_CONSTRUCCION': record['CODIGO_CONSTRUCCION'],
        'CODIGO_RESTO': record['CODIGO_RESTO'],
        'CLASE_PREDIO': record['CLASE_PREDIO'],
        'CHIP': record['CHIP'],
        'PORCENTAJE_COPROPIEDAD': record['PORCENTAJE_COPROPIEDAD'],
        'P.MATRICULA': record['MATRICULA'],
        'CODIGO_ESTRATO': '',  # Placeholder
        'AREA_TERRENO': record['AREA_TERRENO'],
        'AREA_CONSTRUIDA': record['AREA_CONSTRUIDA'],
        'VALOR_REFERENCIA': record['VALOR_REFERENCIA'],
        'VIGENCIA': '',  # Placeholder
        'VALOR_AVALUO': record['VALOR_AVALUO'],
        'DIRECCION_REAL': record['DIRECCION_REAL'],
        'TIPO_PROPIEDAD': tipo_propiedad,
        'DESCRIPCION_TIPO_PROPIEDAD': descripcion_tipo_propiedad,
        'PERIODO': '2023',  # Placeholder
        'AT_VM2': '',  # Placeholder
        'AC_VM2': '',  # Placeholder
        'LOTLOTE_ID': ''

    }
    
    return transformed

def fix_codes_and_generate_loteid(dataset):
    dataset['CODIGO_BARRIO'] = dataset['CODIGO_BARRIO'].astype(str)
    dataset['CODIGO_MANZANA'] = dataset['CODIGO_MANZANA'].astype(str)
    dataset['CODIGO_PREDIO'] = dataset['CODIGO_PREDIO'].astype(str)

    for index, row in dataset.iterrows():
        if len(row['CODIGO_BARRIO']) == 4:
            dataset.at[index,'CODIGO_BARRIO'] = '00{}'.format(row['CODIGO_BARRIO'])
        else:
            pass

        if len(row['CODIGO_MANZANA']) == 1:
            dataset.at[index,'CODIGO_MANZANA'] = '00{}'.format(row['CODIGO_MANZANA'])
        elif len(row['CODIGO_MANZANA']) == 2:
            dataset.at[index,'CODIGO_MANZANA'] = '0{}'.format(row['CODIGO_MANZANA'])
        else:
            pass

        if len(row['CODIGO_PREDIO']) == 1:
            dataset.at[index,'CODIGO_PREDIO'] = '00{}'.format(row['CODIGO_PREDIO'])
        elif len(row['CODIGO_PREDIO']) == 2:
            dataset.at[index,'CODIGO_PREDIO'] = '0{}'.format(row['CODIGO_PREDIO'])
        else:
            pass

    dataset['LOTLOTE_ID'] = (dataset['CODIGO_BARRIO'].astype(str) + dataset['CODIGO_MANZANA'].astype(str) + dataset['CODIGO_PREDIO'].astype(str)).astype(str)
    return dataset

def excetute_2023_transformation():
    # Specify data types for the columns
    column_types = {
        'CODIGO_BARRIO': str,
        'CODIGO_MANZANA': str,
        'CODIGO_PREDIO': str,
        'CODIGO_CONSTRUCCION': str,
        'CODIGO_RESTO': str,
        'CODIGO_USO': str
    }
    
    # Load the data from the second schema with specified data types
    df = pd.read_csv('/dbfs/FileStore/tables/Catastro/2023/SOL0311492_24.txt', delimiter=';', encoding='ISO-8859-1', dtype=column_types)
    
    # Transform the data to match the first schema
    transformed_data = df.apply(transform_record, axis=1)
    transformed_df = pd.DataFrame(list(transformed_data))
    
    trasnformed_nan = transformed_df.copy()

    trasnformed_nan.replace('0', np.nan, inplace=True)
    trasnformed_nan.replace(0, np.nan, inplace=True)
    
    trasnformed_nan = fix_codes_and_generate_loteid(trasnformed_nan)
    
    trasnformed_nan['PORCENTAJE_COPROPIEDAD'] = trasnformed_nan['PORCENTAJE_COPROPIEDAD'].str.replace(',', '.').astype(float)
    trasnformed_nan['AREA_TERRENO'] = trasnformed_nan['AREA_TERRENO'].str.replace(',', '.').astype(float)
    trasnformed_nan['AREA_CONSTRUIDA'] = trasnformed_nan['AREA_CONSTRUIDA'].str.replace(',', '.').astype(float)
    trasnformed_nan['AT_VM2'] = trasnformed_nan['VALOR_AVALUO']/trasnformed_nan['AREA_TERRENO']
    trasnformed_nan['AC_VM2'] = trasnformed_nan['VALOR_AVALUO']/trasnformed_nan['AREA_CONSTRUIDA']
       
    
    # Save the transformed data to a new CSV file
    trasnformed_nan.to_csv('/dbfs/FileStore/tables/Catastro/2023/predios_2023.csv', index=False,  sep=',', quotechar='"')
    return trasnformed_nan


In [0]:
#test_data = excetute_2023_transformation()

In [0]:
#test_data

In [0]:
import pandas as pd
import numpy as np

def transform_record_2(record):

    transformed = {
        '_c0':0,
        'CODIGO_USO': record['USO'],
        'DESCRIPCION_USO': record['DESCRIPCION_USO'],
        'PUNTAJE': '',  # Placeholder
        'EDAD': '',  # Placeholder
        'PERIODO': '2023',  # Placeholder,
        'CHIP':record['CHIP']
    }
    
    return transformed


def excetute_2023_transformation_part_2():
    
    column_types = {
        'SECTOR': str,
        'USO': str,
        'DESCRIPCION_USO': str,
        'CHIP': str,
        'MATRICULA': str,
        'CLASE_PREDIO': str
    }    
    
    # Load the data from the second schema
    df = pd.read_csv('/dbfs/FileStore/tables/Catastro/2023/SOL0311492_24_ph1.txt', delimiter=';', quotechar='\'', encoding='ISO-8859-1',dtype=column_types, on_bad_lines='skip')
    df = pd.concat([df,pd.read_csv('/dbfs/FileStore/tables/Catastro/2023/SOL0311492_24_ph2.txt', delimiter=';', quotechar='\'', encoding='ISO-8859-1',dtype=column_types, on_bad_lines='skip')])
    df = pd.concat([df,pd.read_csv('/dbfs/FileStore/tables/Catastro/2023/SOL0311492_24_nph1.txt', delimiter=';', quotechar='\'', encoding='ISO-8859-1',dtype=column_types, on_bad_lines='skip')])
    df = pd.concat([df,pd.read_csv('/dbfs/FileStore/tables/Catastro/2023/SOL0311492_24_nph2.txt', delimiter=';', quotechar='\'', encoding='ISO-8859-1',dtype=column_types, on_bad_lines='skip')])
    
    # Transform the data to match the first schema
    transformed_data = df.apply(transform_record_2, axis=1)
    transformed_df = pd.DataFrame(list(transformed_data))
    
    column_types = {
        '_c0': str,
        'CODIGO_LOCALIDAD': str,
        'NOMBRE_LOCALIDAD': str,
        'CODIGO_BARRIO': str,
        'CODIGO_MANZANA': str,
        'CODIGO_PREDIO': str,
        'CODIGO_CONSTRUCCION': str,
        'CODIGO_RESTO': str,
        'CLASE_PREDIO': str,
        'CHIP': str,
        'PORCENTAJE_COPROPIEDAD': str,
        'P.MATRICULA': str,
        'CODIGO_ESTRATO': str,
        'AREA_TERRENO': str,
        'AREA_CONSTRUIDA': str,
        'VALOR_REFERENCIA': str,
        'VIGENCIA': str,
        'VALOR_AVALUO': str,
        'DIRECCION_REAL': str,
        'TIPO_PROPIEDAD': int,
        'DESCRIPCION_TIPO_PROPIEDAD': str,
        'PERIODO': int,
        'AT_VM2': str,
        'AC_VM2': str,
        'LOTLOTE_ID': str
    }    
    
    transformed_nan = pd.read_csv('/dbfs/FileStore/tables/Catastro/2023/predios_2023.csv', sep=',', quotechar='"', dtype=column_types, on_bad_lines='skip')
    
    # Replace commas with dots in the columns that are supposed to be float
    float_columns = ['PORCENTAJE_COPROPIEDAD', 'AREA_TERRENO', 'AREA_CONSTRUIDA', 'VALOR_REFERENCIA', 'VALOR_AVALUO', 'AT_VM2', 'AC_VM2']

    for col in float_columns:
        transformed_nan[col] = transformed_nan[col].str.replace(',', '.').astype(float)

    # Convert other columns to their appropriate data types based on the column_types dictionary
    for col, dtype in column_types.items():
        if col not in float_columns:
            transformed_nan[col] = transformed_nan[col].astype(dtype)
    
    transformed_df = pd.merge(left=transformed_df, right=transformed_nan[['LOTLOTE_ID','CHIP','CODIGO_BARRIO','CODIGO_MANZANA','CODIGO_PREDIO','CODIGO_CONSTRUCCION','CODIGO_RESTO']], on=['CHIP'], how='left')
    del(transformed_df['CHIP'])
    # Save the transformed data to a new CSV file
    transformed_df.to_csv('/dbfs/FileStore/tables/Catastro/2023/usos_2023.csv', index=False, sep=',', quotechar='"')
    return transformed_df

In [0]:
#test_data_2 = excetute_2023_transformation_part_2()

In [0]:
#test_data_2

In [0]:
import os

# Define the function to list files in a directory
def list_files(directory):
    files = []
    for file in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, file)):
            files.append(file)
    return files

# Specify the directory path
directory_path = "/dbfs/FileStore/tables/Catastro/2022/"

# Call the function to list files in the specified directory
file_list = list_files(directory_path)

# Print the list of files
print(file_list)

['ph_2022.csv', 'predios_2022.csv', 'usos_2022.csv']


In [0]:
test = pd.read_csv('/dbfs/FileStore/tables/Catastro/2022/predios_2022.csv')
test.columns = test.columns.str.strip()

In [0]:
unique_combinations = test[['CODIGO_LOCALIDAD','NOMBRE_LOCALIDAD']].drop_duplicates()
for index, row in unique_combinations.iterrows():
    print(row['CODIGO_LOCALIDAD'], row['NOMBRE_LOCALIDAD'])

7 BOSA
11 SUBA
1 USAQUEN
19 CIUDAD BOLIVAR
2 CHAPINERO
5 USME
8 KENNEDY
16 PUENTE ARANDA
4 SAN CRISTOBAL
12 BARRIOS UNIDOS
3 SANTA FE
10 ENGATIVA
18 RAFAEL URIBE URIBE
14 LOS MARTIRES
9 FONTIBON
6 TUNJUELITO
17 LA CANDELARIA
13 TEUSAQUILLO
15 ANTONIO NARINO
20 SUMAPAZ


In [0]:
def excecute_2024_predios_transformation(input_csv_path, output_csv_path):
    # Load the original dataset
    column_types = {
        'nombre_Localidad': str,
        'codigo_catastral': str,
        'chip': str,
        'matricula_inmobiliaria': str,
        'direccion_real': str,
        'coordenada_x': float,
        'coordenada_y': float,
        'area_terreno': float,  # Note: 'area_terreno' appears twice in your list. Ensure this is not a mistake.
        'tipo_propiedad': str,
        'estrato': str,
        'area_construida': float,
        'area_comun_terreno': float,
        'area_privada_terreno': float,
        'clase_predio': str,
        'codigo_destino': str,
        'max_num_pisos': str,
        'porcentaje_copropiedad': float,
        'valor_referencia': float,
        'valor_avaluo': float
    }
    df = pd.read_csv(input_csv_path, delimiter=';', quotechar='\'', encoding='ISO-8859-1',dtype=column_types, on_bad_lines='skip')
    df.columns = df.columns.str.strip()
    # Explicitly convert 'codigo_catastral' to string to ensure consistent handling
    df['codigo_catastral'] = df['codigo_catastral'].astype(str)

    
    # Transform the dataset
    df_transformed = pd.DataFrame({
        'CODIGO_LOCALIDAD': df['nombre_Localidad'],
        'NOMBRE_LOCALIDAD': df['nombre_Localidad'].apply(lambda x: describe_nombre_localidad(x)),
        'CODIGO_BARRIO': df['codigo_catastral'].apply(lambda x: extract_codigo_barrio(x)),
        'CODIGO_MANZANA': df['codigo_catastral'].apply(lambda x: extract_codigo_manzana(x)),
        'CODIGO_PREDIO': df['codigo_catastral'].apply(lambda x: extract_codigo_predio(x)),
        'CODIGO_CONSTRUCCION': df['codigo_catastral'].apply(lambda x: extract_codigo_construccion(x)),
        'CODIGO_RESTO': df['codigo_catastral'].apply(lambda x: extract_codigo_resto(x)),
        'CLASE_PREDIO': df['clase_predio'],
        'CHIP': df['chip'],
        'PORCENTAJE_COPROPIEDAD': df['porcentaje_copropiedad'],
        'P.MATRICULA': df['matricula_inmobiliaria'],
        'CODIGO_ESTRATO': df['estrato'],
        'AREA_TERRENO': df['area_terreno'],
        'AREA_CONSTRUIDA': df['area_construida'],
        'VALOR_REFERENCIA': df['valor_referencia'],
        'VIGENCIA': '2022',  # Assuming a fixed value for all rows
        'VALOR_AVALUO': df['valor_avaluo'],
        'DIRECCION_REAL': df['direccion_real'],
        'TIPO_PROPIEDAD': df['tipo_propiedad'],
        'DESCRIPCION_TIPO_PROPIEDAD': df['tipo_propiedad'].apply(lambda x: describe_tipo_propiedad(x)),
        'PERIODO': '2024',  # Assuming a fixed value for all rows
        'AT_VM2': calculate_at_vm2(df['area_terreno'], df['valor_avaluo']),
        'AC_VM2': calculate_ac_vm2(df['area_construida'], df['valor_avaluo']),
        'LOTLOTE_ID': df['codigo_catastral'].apply(lambda x: extract_lotlote_id(x))
    })
    
    # Save the transformed dataset to a new CSV file
    df_transformed.to_csv(output_csv_path, index=False)

    return df_transformed

def extract_codigo_barrio(x):
    # Implement your extraction logic here
    return x[0:6]

def extract_codigo_manzana(x):
    # Implement your extraction logic here
    return x[6:9]

def extract_codigo_predio(x):
    # Implement your extraction logic here
    return x[9:12]

def extract_codigo_construccion(x):
    # Implement your extraction logic here
    return x[12:15]

def extract_codigo_resto(x):
    # Implement your extraction logic here
    return x[15:18]

def describe_nombre_localidad(x):
    # Map numerical codes to descriptions
    descriptions = {
        '1': 'USAQUEN',
        '2': 'CHAPINERO',
        '3': 'SANTA FE',
        '4': 'SAN CRISTOBAL',
        '5': 'USME',
        '6': 'TUNJUELITO',
        '7': 'BOSA',
        '8': 'KENNEDY',
        '9': 'FONTIBON',
        '10': 'ENGATIVA',
        '11': 'SUBA',
        '12': 'BARRIOS UNIDOS',
        '13': 'TEUSAQUILLO',
        '14': 'LOS MARTIRES',
        '15': 'ANTONIO NARINO',
        '16': 'PUENTE ARANDA',
        '17': 'LA CANDELARIA',
        '18': 'RAFAEL URIBE URIBE',
        '19': 'CIUDAD BOLIVAR',
        '20': 'SUMAPAZ'
    }
    return descriptions.get(x, 'Unknown')

def describe_tipo_propiedad(x):
    # Map numerical codes to descriptions
    descriptions = {
        '1':'OFICIAL',
        '2':'DISTRITAL',
        '3':'RELIGIOSO',
        '4':'EMBAJADA',
        '5':'PARQUES',
        '6':'PARTICULAR',
        '7':'MIXTO',
        '8':'OTROS'
    }
    return descriptions.get(x, 'Unknown')

def calculate_at_vm2(area_terreno, valor_avaluo):
    # Calculate the value per square meter for the land area
    return valor_avaluo / area_terreno

def calculate_ac_vm2(area_construida, valor_avaluo):
    # Calculate the value per square meter for the constructed area
    return valor_avaluo / area_construida

def extract_lotlote_id(x):
    # Implement your extraction logic here
    return x[:12]



In [0]:
test_df_3 = excecute_2024_predios_transformation('/dbfs/FileStore/tables/Catastro/2024/SOL0311009_24_Metro.txt', '/dbfs/FileStore/tables/Catastro/2024/predios_2024.csv')

In [0]:
test_df_3

,CODIGO_LOCALIDAD,NOMBRE_LOCALIDAD,CODIGO_BARRIO,CODIGO_MANZANA,CODIGO_PREDIO,CODIGO_CONSTRUCCION,CODIGO_RESTO,CLASE_PREDIO,CHIP,PORCENTAJE_COPROPIEDAD,P.MATRICULA,CODIGO_ESTRATO,AREA_TERRENO,AREA_CONSTRUIDA,VALOR_REFERENCIA,VIGENCIA,VALOR_AVALUO,DIRECCION_REAL,TIPO_PROPIEDAD,DESCRIPCION_TIPO_PROPIEDAD,PERIODO,AT_VM2,AC_VM2,LOTLOTE_ID
0,16,PUENTE ARANDA,004401,151,600,102,001,P,AAA0209OMKC,16.56,050S40516184,3,23.18,52.55,2970000.0,2022,155383000.0,CL 20 SUR 51F 63 AP 201,6,PARTICULAR,2024,6.703322e+06,2.956860e+06,004401151600
1,1,USAQUEN,008412,160,600,104,004,P,AAA0213MWXR,3.07,050N20537012,6,26.34,59.11,4850000.0,2022,328792000.0,CL 104 19A 64 AP 404,6,PARTICULAR,2024,1.248261e+07,5.562375e+06,008412160600
2,1,USAQUEN,008412,160,600,105,002,P,AAA0213MWZM,7.47,050N20537014,6,64.09,143.48,4850000.0,2022,782258000.0,CL 104 19A 64 AP 502,6,PARTICULAR,2024,1.220562e+07,5.452035e+06,008412160600
3,8,KENNEDY,004542,342,900,000,000,N,AAA0049WPFT,NaN,050S00120456,2,120.00,270.74,2470000.0,2022,263664000.0,CL 35A SUR 68B 53,6,PARTICULAR,2024,2.197200e+06,9.738642e+05,004542342900
4,1,USAQUEN,008519,270,100,105,011,P,AAA0114XDEA,0.58,050N01004000,4,35.74,60.60,3000000.0,2022,175915000.0,AK 19 160 05 IN 3 AP 503,6,PARTICULAR,2024,4.922076e+06,2.902888e+06,008519270100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819049,19,CIUDAD BOLIVAR,104134,021,909,500,000,N,AAA0185JDSK,NaN,050S00000000,1,0.00,36.68,NaN,2022,935000.0,CL 70IBIS SUR 26 01 MJ 112,6,PARTICULAR,2024,inf,2.549073e+04,104134021909
2819050,19,CIUDAD BOLIVAR,104134,021,901,000,000,N,AAA0185HZSK,NaN,050S00000000,1,0.00,36.00,NaN,2022,14001000.0,CL 70IBIS SUR 26 01 MJ 39,6,PARTICULAR,2024,inf,3.889167e+05,104134021901
2819051,19,CIUDAD BOLIVAR,104134,021,300,000,000,N,AAA0143CMHY,NaN,050S40222373,0,611.30,0.00,NaN,2022,7302000.0,DG 77 SUR 25A 13 IN 09,6,PARTICULAR,2024,1.194504e+04,inf,104134021300
2819052,19,CIUDAD BOLIVAR,002595,701,200,100,000,N,AAA0284NBJH,NaN,050S00000000,1,0.00,36.00,600000.0,2022,966000.0,CL 91B S 18F 72 MJ,6,PARTICULAR,2024,inf,2.683333e+04,002595701200


In [0]:
def fix_codes_and_generate_loteid(dataset):
    dataset['CODIGO_BARRIO'] = dataset['CODIGO_BARRIO'].astype(str)
    dataset['CODIGO_MANZANA'] = dataset['CODIGO_MANZANA'].astype(str)
    dataset['CODIGO_PREDIO'] = dataset['CODIGO_PREDIO'].astype(str)

    for index, row in dataset.iterrows():
        if len(row['CODIGO_BARRIO']) == 4:
            dataset.at[index,'CODIGO_BARRIO'] = '00{}'.format(row['CODIGO_BARRIO'])
        else:
            pass

        if len(row['CODIGO_MANZANA']) == 1:
            dataset.at[index,'CODIGO_MANZANA'] = '00{}'.format(row['CODIGO_MANZANA'])
        elif len(row['CODIGO_MANZANA']) == 2:
            dataset.at[index,'CODIGO_MANZANA'] = '0{}'.format(row['CODIGO_MANZANA'])
        else:
            pass

        if len(row['CODIGO_PREDIO']) == 1:
            dataset.at[index,'CODIGO_PREDIO'] = '00{}'.format(row['CODIGO_PREDIO'])
        elif len(row['CODIGO_PREDIO']) == 2:
            dataset.at[index,'CODIGO_PREDIO'] = '0{}'.format(row['CODIGO_PREDIO'])
        else:
            pass

    dataset['LOTLOTE_ID'] = (dataset['CODIGO_BARRIO'].astype(str) + dataset['CODIGO_MANZANA'].astype(str) + dataset['CODIGO_PREDIO'].astype(str)).astype(str)
    return dataset

def excecute_2024_ph_transformation(input_file, output_file):

    # Load the data into a DataFrame
    df = pd.read_csv(input_file, sep='|', dtype=str)
    df.columns = df.columns.str.strip()
    df.columns = ['CODIGO_BARRIO', 'CODIGO_MANZANA', 'CODIGO_PREDIO', 'NUMERO_PISOS', 'AREA_COMUN_CONSTRUIDA', 'AREA_PRIVADA_CONSTRUIDA', 'NUMERO_UNIDADES_PH']

    # Convert numeric columns to float or int as needed
    df['NUMERO_PISOS'] = df['NUMERO_PISOS'].astype(float)
    df['NUMERO_UNIDADES_PH'] = df['NUMERO_UNIDADES_PH'].astype(int)

    # Add PERIODO column
    df['PERIODO'] = 2024

    # Create LOTLOTE_ID column
    df = fix_codes_and_generate_loteid(df)

    # Adjust formatting for AREA columns
    df['AREA_COMUN_CONSTRUIDA'] = df['AREA_COMUN_CONSTRUIDA'].str.replace('.', ',', regex=False)
    df['AREA_PRIVADA_CONSTRUIDA'] = df['AREA_PRIVADA_CONSTRUIDA'].str.replace('.', ',', regex=False)

    # Reorder columns as needed
    df = df[['CODIGO_BARRIO', 'CODIGO_MANZANA', 'CODIGO_PREDIO', 'NUMERO_PISOS', 'AREA_COMUN_CONSTRUIDA', 'AREA_PRIVADA_CONSTRUIDA', 'NUMERO_UNIDADES_PH', 'PERIODO', 'LOTLOTE_ID']]

    # Save the transformed dataset to a new CSV file
    df.to_csv(output_file, index=False)

    return df


In [0]:
test_df_4 = excecute_2024_ph_transformation('/dbfs/FileStore/tables/Catastro/2024/SOL0311009_24_CARATULA_PH_2024.txt','/dbfs/FileStore/tables/Catastro/2024/ph_2024.csv')

In [0]:
test_df_4

,CODIGO_BARRIO,CODIGO_MANZANA,CODIGO_PREDIO,NUMERO_PISOS,AREA_COMUN_CONSTRUIDA,AREA_PRIVADA_CONSTRUIDA,NUMERO_UNIDADES_PH,PERIODO,LOTLOTE_ID
0,001101,002,061,2.0,"30,2",2200,40,2024,001101002061
1,001101,006,045,6.0,0,"4508,73",98,2024,001101006045
2,001101,006,058,15.0,0,"53776,39",1342,2024,001101006058
3,001101,009,001,5.0,"7072,1","23123,6",629,2024,001101009001
4,001101,009,002,4.0,0,"2980,46",86,2024,001101009002
...,...,...,...,...,...,...,...,...,...
52415,205322,024,001,6.0,0,"18759,22",440,2024,205322024001
52416,205322,024,002,6.0,0,"18759,22",440,2024,205322024002
52417,205322,025,001,6.0,0,"34821,6",792,2024,205322025001
52418,205322,026,001,6.0,0,31656,720,2024,205322026001


In [0]:
def fix_codes_and_generate_loteid(dataset):
    dataset['CODIGO_BARRIO'] = dataset['CODIGO_BARRIO'].astype(str)
    dataset['CODIGO_MANZANA'] = dataset['CODIGO_MANZANA'].astype(str)
    dataset['CODIGO_PREDIO'] = dataset['CODIGO_PREDIO'].astype(str)

    for index, row in dataset.iterrows():
        if len(row['CODIGO_BARRIO']) == 4:
            dataset.at[index,'CODIGO_BARRIO'] = '00{}'.format(row['CODIGO_BARRIO'])
        else:
            pass

        if len(row['CODIGO_MANZANA']) == 1:
            dataset.at[index,'CODIGO_MANZANA'] = '00{}'.format(row['CODIGO_MANZANA'])
        elif len(row['CODIGO_MANZANA']) == 2:
            dataset.at[index,'CODIGO_MANZANA'] = '0{}'.format(row['CODIGO_MANZANA'])
        else:
            pass

        if len(row['CODIGO_PREDIO']) == 1:
            dataset.at[index,'CODIGO_PREDIO'] = '00{}'.format(row['CODIGO_PREDIO'])
        elif len(row['CODIGO_PREDIO']) == 2:
            dataset.at[index,'CODIGO_PREDIO'] = '0{}'.format(row['CODIGO_PREDIO'])
        else:
            pass

    dataset['LOTLOTE_ID'] = (dataset['CODIGO_BARRIO'].astype(str) + dataset['CODIGO_MANZANA'].astype(str) + dataset['CODIGO_PREDIO'].astype(str)).astype(str)
    return dataset

def excecute_2024_usos_transformation(input_file, output_file):

    # Load the data into a DataFrame
    df = pd.read_csv(input_file, sep='|', dtype=str)
    df.columns = df.columns.str.strip()
    
    # Convert numeric columns to the appropriate types
    df['PUNTAJE'] = pd.to_numeric(df['PUNTAJE'], errors='coerce').astype('Int64')
    df['EDAD'] = pd.to_numeric(df['EDAD'], errors='coerce').astype('Int64')     

    # Add PERIODO column
    df['PERIODO'] = 2024

    # Create LOTLOTE_ID column
    df = fix_codes_and_generate_loteid(df)

    # Reorder columns as needed
    df = df[['CODIGO_BARRIO', 'CODIGO_MANZANA', 'CODIGO_PREDIO', 'CODIGO_CONSTRUCCION', 'CODIGO_RESTO', 'CODIGO_USO', 'DESCRIPCION_USO', 'PUNTAJE', 'EDAD', 'PERIODO', 'LOTLOTE_ID']]

    # Save the transformed dataset to a new CSV file
    df.to_csv(output_file, index=False)

    return df


In [0]:
test_df_5 = excecute_2024_usos_transformation('/dbfs/FileStore/tables/Catastro/2024/SOL0311009_24_USOS_2024.TXT','/dbfs/FileStore/tables/Catastro/2024/usos_2024.csv')

In [0]:
test_df_5

,CODIGO_BARRIO,CODIGO_MANZANA,CODIGO_PREDIO,CODIGO_CONSTRUCCION,CODIGO_RESTO,CODIGO_USO,DESCRIPCION_USO,PUNTAJE,EDAD,PERIODO,LOTLOTE_ID
0,003102,018,018,000,00000,004,CORREDOR COMERCIAL NPH O HASTA 3 UNID PH,62,1995,2024,003102018018
1,003102,018,018,000,00000,020,OFICINAS Y CONSULTORIOS NPH,58,1995,2024,003102018018
2,003102,009,009,001,01001,040,CORREDOR COMERCIAL PH,50,1950,2024,003102009009
3,003102,010,019,001,01002,040,CORREDOR COMERCIAL PH,51,1972,2024,003102010019
4,003102,015,011,000,00000,004,CORREDOR COMERCIAL NPH O HASTA 3 UNID PH,59,1966,2024,003102015011
...,...,...,...,...,...,...,...,...,...,...,...
3263531,104134,002,013,000,00000,NaN,NaN,<NA>,<NA>,2024,104134002013
3263532,104134,002,007,000,00009,NaN,NaN,<NA>,<NA>,2024,104134002007
3263533,005203,003,007,000,00009,NaN,NaN,<NA>,<NA>,2024,005203003007
3263534,008539,004,019,000,00000,NaN,NaN,<NA>,<NA>,2024,008539004019


In [0]:
with open('/dbfs/FileStore/tables/Catastro/2022/usos_2022.csv', 'r') as file:
    for i in range(4):
        print(file.readline())

,CODIGO_BARRIO,CODIGO_MANZANA,CODIGO_PREDIO,CODIGO_CONSTRUCCION,CODIGO_RESTO,CODIGO_USO,DESCRIPCION_USO,PUNTAJE,EDAD,PERIODO,LOTLOTE_ID

0,006205,029,006,0,0,25,BODEGAS DE ALMACENAMIENTO NPH,72,1982,2022,006205029006

1,009233,060,050,1,1001,38,HABITACIONAL EN PROPIEDAD HORIZONTAL,41,1997,2022,009233060050

2,006205,029,009,0,0,10,INDUSTRIA MEDIANA,74,1982,2022,006205029009



In [0]:
with open('/dbfs/FileStore/tables/Catastro/2024/SOL0311009_24_USOS_2024.TXT', 'r') as file:
    for i in range(4):
        print(file.readline())

CODIGO_BARRIO|CODIGO_MANZANA|CODIGO_PREDIO|CODIGO_CONSTRUCCION|CODIGO_RESTO|CODIGO_USO|DESCRIPCION_USO|PUNTAJE|EDAD

003102|18|18|000|00000|004|CORREDOR COMERCIAL NPH O HASTA 3 UNID PH|62|1995     

003102|18|18|000|00000|020|OFICINAS Y CONSULTORIOS NPH|58|1995                  

003102|09|09|001|01001|040|CORREDOR COMERCIAL PH|50|1950                        

